In [1]:
import paramiko
import zipfile
import pickle
import os
import pandas as pd
from datetime import datetime, timedelta

def download_and_extract_all_files(hostname, port, username, password, remote_folder, local_folder):
    # Create an SSH client
    ssh = paramiko.SSHClient()

    try:
        # Automatically add the server's host key
        ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())

        # Connect to the SSH server
        ssh.connect(hostname, port, username, password)

        # Open an SFTP session
        with ssh.open_sftp() as sftp:
            # Change to the remote folder
            sftp.chdir(remote_folder)

            # List files in the remote folder
            files = sftp.listdir()

            # Dictionary to store dataframes for each date
            date_dataframes = {}

            # Iterate through files and download/extract for each date
            for file in files:
                if file.startswith("UnderlyingEOD") and not "Summaries" in file:
                    date_format = file.split("_")[-1].split(".")[0]  # Extract date from the filename
                    remote_path = os.path.join(remote_folder, file).replace("\\", "/")
                    local_path = os.path.join(local_folder, file).replace("\\", "/")

                    # Download the file
                    sftp.get(remote_path, local_path)

                    # Extract files from the downloaded ZIP file
                    extract_zip(local_path, local_folder)

                    # Read the CSV file into a DataFrame
                    csv_file = [csv for csv in os.listdir(local_folder) if csv.endswith('.csv')]
                    if csv_file:
                        csv_file_path = os.path.join(local_folder, csv_file[0])
                        eod_df = pd.read_csv(csv_file_path)

                        # Store the DataFrame in the dictionary with the date as the key
                        date_dataframes[date_format] = eod_df

            # Clean up: Remove downloaded ZIP files
            for file in os.listdir(local_folder):
                file_path = os.path.join(local_folder, file)
                if file_path.endswith(".zip"):
                    os.remove(file_path)

    except Exception as e:
        print(f"Error: {e}")

    finally:
        # Close the SSH connection
        ssh.close()

    return date_dataframes

def extract_zip(zip_filename, extraction_folder):
    with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
        # Extract all files in the ZIP archive to the extraction folder
        zip_ref.extractall(extraction_folder)

# SFTP server details
sftp_hostname = "sftp.datashop.livevol.com"
sftp_port = 22  # Change the port if your SFTP server uses a different port
sftp_username = "nan2_lehigh_edu"
sftp_password = "PAIndex2023!"
sftp_remote_folder = "/subscriptions/order_000046197/item_000053507"  # Change to the actual path on the SFTP server
local_download_folder = os.path.join(os.getcwd(), "input")  # Set to "input" folder inside the current directory

# Create the local download folder if it doesn't exist
os.makedirs(local_download_folder, exist_ok=True)

# Download all files and extract for each date
date_dataframes = download_and_extract_all_files(sftp_hostname, sftp_port, sftp_username, sftp_password, sftp_remote_folder, local_download_folder)

In [2]:
local_download_folder = os.path.join(os.getcwd(), "input")  # Set to "input" folder inside the current directory

# Create a dictionary to store dataframes for each date
date_dataframes = {}

# Loop through all files in the "input" folder
for file in os.listdir(local_download_folder):
    if file.startswith("UnderlyingEOD_") and file.endswith(".csv"):
        # Extract date from the filename
        date_format = file.split("_")[-1].split(".")[0]

        # Create the full file path
        csv_file_path = os.path.join(local_download_folder, file)

        # Read the CSV file into a DataFrame
        eod_df = pd.read_csv(csv_file_path)

        # Store the DataFrame in the dictionary with the date as the key
        date_dataframes[date_format] = eod_df

        # After using the CSV file, optionally delete it
        try:
            os.remove(csv_file_path)
            print(f"Deleted: {csv_file_path}")
        except FileNotFoundError:
            print(f"File not found: {csv_file_path}")
        except Exception as e:
            print(f"Error deleting file: {e}")

# Now, you can access each date's DataFrame using date_dataframes[date_format]
date_dataframes.pop(date_format)
output_file_path = os.path.join(os.getcwd(), r"input\date_dataframes.pkl")
with open(output_file_path, 'wb') as output_file:
    pickle.dump(date_dataframes, output_file)

print(f"Saved date_dataframes to: {output_file_path}")

Deleted: /Users/rik/Desktop/PA-Index/input/UnderlyingEOD_2024-05-28.csv
Deleted: /Users/rik/Desktop/PA-Index/input/UnderlyingEOD_2024-05-29.csv
Deleted: /Users/rik/Desktop/PA-Index/input/UnderlyingEOD_2024-05-17.csv
Deleted: /Users/rik/Desktop/PA-Index/input/UnderlyingEOD_2024-06-07.csv
Deleted: /Users/rik/Desktop/PA-Index/input/UnderlyingEOD_2024-06-13.csv
Deleted: /Users/rik/Desktop/PA-Index/input/UnderlyingEOD_2024-06-12.csv
Deleted: /Users/rik/Desktop/PA-Index/input/UnderlyingEOD_2024-06-06.csv
Deleted: /Users/rik/Desktop/PA-Index/input/UnderlyingEOD_2024-06-10.csv
Deleted: /Users/rik/Desktop/PA-Index/input/UnderlyingEOD_2024-06-04.csv
Deleted: /Users/rik/Desktop/PA-Index/input/UnderlyingEOD_2024-06-05.csv
Deleted: /Users/rik/Desktop/PA-Index/input/UnderlyingEOD_2024-06-11.csv
Deleted: /Users/rik/Desktop/PA-Index/input/UnderlyingEOD_2024-06-14.csv
Deleted: /Users/rik/Desktop/PA-Index/input/UnderlyingEOD_2024-06-17.csv
Deleted: /Users/rik/Desktop/PA-Index/input/UnderlyingEOD_2024-06

In [3]:
#date_dataframes['2023-11-21']

In [4]:
df = pd.read_excel("input/RAY as of Oct 23 20231_PA.xlsx")
df.columns = df.columns.str.rstrip('\n')
df['Ticker'] = df['Ticker'].str.split(' ', n=1, expand=True)[0]
df['Ticker'] = df['Ticker'].str.replace(' ', '')  # Remove spaces
# Create float_df by selecting the first 100 rows of "Ticker" and "Equity Float" columns
float_df = df[['Ticker', 'Equity Float']].head(100)
float_df

,Ticker,Equity Float
0,CMCSA,4085497600
1,APD,221426736
2,PNC,395541408
3,HSY,147389968
4,KHC,786508992
...,...,...
95,CCNE,20148332
96,MLYS,18351756
97,LMB,9479473
98,MPB,14668140


In [5]:
# Create an empty DataFrame to store market cap DataFrames for each date
eod_market_cap_pivot = pd.DataFrame()

# Extract unique dates from date_dataframes dictionary
unique_dates = list(date_dataframes.keys())

# Iterate over unique dates
for date_format in unique_dates[:-1]:
    # Access the corresponding DataFrame from date_dataframes
    eod_df = date_dataframes[date_format]

    # Merge dataframes on the common column 'Ticker' and 'underlying_symbol'
    merged_df = pd.merge(float_df, eod_df, left_on='Ticker', right_on='underlying_symbol')

    # Perform the multiplication and rename the column to 'Market Cap'
    merged_df['Market Cap'] = merged_df['Equity Float'] * merged_df['close']

    # Use pivot_table to create a multi-level DataFrame
    eod_market_cap_daily = merged_df.pivot_table(index='quote_date', columns='Ticker', values='Market Cap', aggfunc='sum')

    # Add a new column for the sum of market caps for each date
    #eod_market_cap_daily['close index market cap'] = eod_market_cap_daily.sum(axis=1)

    # Append the daily market cap DataFrame to the main DataFrame
    eod_market_cap_pivot = pd.concat([eod_market_cap_pivot, eod_market_cap_daily])

# Rename the index to 'Date'
eod_market_cap_pivot = eod_market_cap_pivot.rename_axis(index='Date')

# Display the resulting pivot table
eod_market_cap_pivot.index = pd.to_datetime(eod_market_cap_pivot.index, errors='coerce')
#eod_market_cap_pivot.index.name = None  # Remove the index name

# Display the resulting pivot table
eod_market_cap_pivot
eod_market_cap_pivot.to_csv("input/LUPA100Mktcap_2023.csv")

In [6]:
eod_market_cap_pivot_new = pd.read_csv("input/LUPA100Mktcap_2023.csv", index_col = "Date")
eod_market_cap_pivot_new

,AA,ACRS,AEO,AHCO,AME,ANSS,APD,ARMK,AUR,AVTR,...,VSH,VSTS,VTRS,WAB,WCC,WMK,WST,WTRG,X,YORW
Date,,,,,,,,,,,,,,,,,,,,,
2024-05-28,7.631862e+09,67194867.60,4.328354e+09,7.947732e+08,3.867254e+10,2.835554e+10,5.861387e+10,8.260849e+09,1.691698e+09,1.592330e+10,...,2.996602e+09,1.512480e+09,1.240677e+10,2.992027e+10,9.375366e+09,1.051814e+09,2.405883e+10,8.848332e+09,7.304796e+09,4.237596e+08
2024-05-29,7.759030e+09,65275014.24,4.350077e+09,7.853397e+08,3.832339e+10,2.786394e+10,5.763517e+10,8.081717e+09,1.698689e+09,1.582256e+10,...,2.942507e+09,1.517691e+09,1.232908e+10,2.970745e+10,8.929930e+09,1.052140e+09,2.370470e+10,8.712688e+09,7.364313e+09,4.171000e+08
2024-05-17,7.285682e+09,75514232.16,4.259564e+09,7.853397e+08,3.831420e+10,2.832011e+10,5.816880e+10,8.645074e+09,1.908403e+09,1.678293e+10,...,3.010440e+09,1.632332e+09,1.311197e+10,2.984755e+10,9.345569e+09,1.117185e+09,2.612778e+10,9.608905e+09,7.130211e+09,4.483533e+08
2024-06-07,7.227396e+09,65275014.24,3.870356e+09,8.808540e+08,3.904925e+10,2.799520e+10,6.189542e+10,8.632094e+09,1.590336e+09,1.612477e+10,...,2.848156e+09,1.597158e+09,1.250836e+10,2.893776e+10,8.908719e+09,1.024100e+09,2.313604e+10,8.959753e+09,7.592464e+09,4.239932e+08
2024-06-13,6.805268e+09,76154183.28,3.781653e+09,9.119060e+08,3.996116e+10,2.806605e+10,6.315533e+10,8.743727e+09,1.747622e+09,1.478832e+10,...,2.894702e+09,1.559379e+09,1.241872e+10,2.898387e+10,8.820338e+09,1.016601e+09,2.471383e+10,8.942798e+09,7.290908e+09,4.252784e+08
2024-06-12,7.011917e+09,74234329.92,3.788894e+09,9.060100e+08,3.990144e+10,2.819395e+10,6.267041e+10,8.790457e+09,1.768593e+09,1.486555e+10,...,2.931814e+09,1.594553e+09,1.270558e+10,2.928891e+10,8.992048e+09,1.024752e+09,2.488024e+10,8.952487e+09,7.388120e+09,4.217734e+08
2024-06-06,7.532954e+09,64635063.12,3.901131e+09,8.745650e+08,3.876671e+10,2.807470e+10,5.974093e+10,8.484115e+09,1.625288e+09,1.616507e+10,...,2.904767e+09,1.619305e+09,1.244860e+10,2.909560e+10,8.861751e+09,1.043500e+09,2.315734e+10,9.124464e+09,7.602384e+09,4.280825e+08
2024-06-10,7.269786e+09,68474769.84,3.759930e+09,8.647384e+08,3.974524e+10,2.795977e+10,6.257077e+10,8.769688e+09,1.639269e+09,1.575876e+10,...,2.849414e+09,1.566544e+09,1.256813e+10,2.903708e+10,9.077903e+09,1.030458e+09,2.350780e+10,8.950064e+09,7.548818e+09,4.232922e+08
2024-06-04,7.418149e+09,65275014.24,3.910182e+09,7.295248e+08,3.833947e+10,2.739808e+10,5.985608e+10,8.307579e+09,1.478488e+09,1.623894e+10,...,2.879606e+09,1.610186e+09,1.257410e+10,2.921974e+10,8.719333e+09,1.047575e+09,2.339392e+10,9.206819e+09,7.556754e+09,4.334568e+08


In [7]:
# Create an empty DataFrame to store market cap DataFrames for each date
eod_market_cap_pivot = pd.DataFrame()

# Extract unique dates from date_dataframes dictionary
unique_dates = list(date_dataframes.keys())

# Iterate over unique dates
for date_format in unique_dates:
    # Access the corresponding DataFrame from date_dataframes
    eod_df = date_dataframes[date_format]

    # Merge dataframes on the common column 'Ticker' and 'underlying_symbol'
    merged_df = pd.merge(float_df, eod_df, left_on='Ticker', right_on='underlying_symbol')

    # Perform the multiplication and rename the column to 'Market Cap'
    merged_df['Market Cap'] = merged_df['Equity Float'] * merged_df['close']

    # Use pivot_table to create a multi-level DataFrame
    eod_market_cap_daily = merged_df.pivot_table(index='quote_date', columns='Ticker', values='Market Cap', aggfunc='sum')

    # Add a new column for the sum of market caps for each date
    eod_market_cap_daily['close index market cap'] = eod_market_cap_daily.sum(axis=1)

    # Append the daily market cap DataFrame to the main DataFrame
    eod_market_cap_pivot = pd.concat([eod_market_cap_pivot, eod_market_cap_daily])

# Rename the index to 'Date'
eod_market_cap_pivot = eod_market_cap_pivot.rename_axis(index='Date')

# Display the resulting pivot table
eod_market_cap_pivot.index = pd.to_datetime(eod_market_cap_pivot.index, errors='coerce')
#eod_market_cap_pivot.index.name = None  # Remove the index name

# Display the resulting pivot table
eod_market_cap_pivot

Ticker,AA,ACRS,AEO,AHCO,AME,ANSS,APD,ARMK,AUR,AVTR,...,VSTS,VTRS,WAB,WCC,WMK,WST,WTRG,X,YORW,close index market cap
Date,,,,,,,,,,,,,,,,,,,,,
2024-05-28,7.631862e+09,67194867.60,4.328354e+09,7.947732e+08,3.867254e+10,2.835554e+10,5.861387e+10,8.260849e+09,1.691698e+09,1.592330e+10,...,1.512480e+09,1.240677e+10,2.992027e+10,9.375366e+09,1.051814e+09,2.405883e+10,8.848332e+09,7.304796e+09,4.237596e+08,9.384647e+11
2024-05-29,7.759030e+09,65275014.24,4.350077e+09,7.853397e+08,3.832339e+10,2.786394e+10,5.763517e+10,8.081717e+09,1.698689e+09,1.582256e+10,...,1.517691e+09,1.232908e+10,2.970745e+10,8.929930e+09,1.052140e+09,2.370470e+10,8.712688e+09,7.364313e+09,4.171000e+08,9.295978e+11
2024-05-17,7.285682e+09,75514232.16,4.259564e+09,7.853397e+08,3.831420e+10,2.832011e+10,5.816880e+10,8.645074e+09,1.908403e+09,1.678293e+10,...,1.632332e+09,1.311197e+10,2.984755e+10,9.345569e+09,1.117185e+09,2.612778e+10,9.608905e+09,7.130211e+09,4.483533e+08,9.565944e+11
2024-06-07,7.227396e+09,65275014.24,3.870356e+09,8.808540e+08,3.904925e+10,2.799520e+10,6.189542e+10,8.632094e+09,1.590336e+09,1.612477e+10,...,1.597158e+09,1.250836e+10,2.893776e+10,8.908719e+09,1.024100e+09,2.313604e+10,8.959753e+09,7.592464e+09,4.239932e+08,9.446506e+11
2024-06-13,6.805268e+09,76154183.28,3.781653e+09,9.119060e+08,3.996116e+10,2.806605e+10,6.315533e+10,8.743727e+09,1.747622e+09,1.478832e+10,...,1.559379e+09,1.241872e+10,2.898387e+10,8.820338e+09,1.016601e+09,2.471383e+10,8.942798e+09,7.290908e+09,4.252784e+08,9.378103e+11
2024-06-12,7.011917e+09,74234329.92,3.788894e+09,9.060100e+08,3.990144e+10,2.819395e+10,6.267041e+10,8.790457e+09,1.768593e+09,1.486555e+10,...,1.594553e+09,1.270558e+10,2.928891e+10,8.992048e+09,1.024752e+09,2.488024e+10,8.952487e+09,7.388120e+09,4.217734e+08,9.431074e+11
2024-06-06,7.532954e+09,64635063.12,3.901131e+09,8.745650e+08,3.876671e+10,2.807470e+10,5.974093e+10,8.484115e+09,1.625288e+09,1.616507e+10,...,1.619305e+09,1.244860e+10,2.909560e+10,8.861751e+09,1.043500e+09,2.315734e+10,9.124464e+09,7.602384e+09,4.280825e+08,9.460539e+11
2024-06-10,7.269786e+09,68474769.84,3.759930e+09,8.647384e+08,3.974524e+10,2.795977e+10,6.257077e+10,8.769688e+09,1.639269e+09,1.575876e+10,...,1.566544e+09,1.256813e+10,2.903708e+10,9.077903e+09,1.030458e+09,2.350780e+10,8.950064e+09,7.548818e+09,4.232922e+08,9.432560e+11
2024-06-04,7.418149e+09,65275014.24,3.910182e+09,7.295248e+08,3.833947e+10,2.739808e+10,5.985608e+10,8.307579e+09,1.478488e+09,1.623894e+10,...,1.610186e+09,1.257410e+10,2.921974e+10,8.719333e+09,1.047575e+09,2.339392e+10,9.206819e+09,7.556754e+09,4.334568e+08,9.472297e+11


In [8]:
eod_market_cap_pivot["CMCSA"]
#eod_market_cap_pivot[["CMCSA"]]

Date
2024-05-28    1.575368e+11
2024-05-29    1.550855e+11
2024-05-17    1.604783e+11
2024-06-07    1.592118e+11
2024-06-13    1.534717e+11
2024-06-12    1.544727e+11
2024-06-06    1.596204e+11
2024-06-10    1.585173e+11
2024-06-04    1.616019e+11
2024-06-05    1.598247e+11
2024-06-11    1.565971e+11
2024-06-14    1.529202e+11
2024-06-17    1.523482e+11
2024-06-03    1.616631e+11
2024-05-21    1.602332e+11
2024-05-20    1.601924e+11
2024-05-22    1.588441e+11
2024-05-23    1.580679e+11
2024-05-30    1.588033e+11
2024-05-24    1.574142e+11
Name: CMCSA, dtype: float64

In [9]:
# # Create the new columns
# eod_market_cap_pivot['gross_change_mcap'] = eod_market_cap_pivot['close index market cap'].diff().fillna(0)

# # Calculate mkt_cap_deleted_stock
# eod_market_cap_pivot['mkt_cap_deleted_stock'] = eod_market_cap_pivot.apply(
#     lambda row: row['close index market cap'] if pd.isna(row['close index market cap']) or row['close index market cap'] == 0 else 0,
#     axis=1
# )

# # Fill NaNs with 0 in the mkt_cap_deleted_stock column
# eod_market_cap_pivot['mkt_cap_deleted_stock'] = eod_market_cap_pivot['mkt_cap_deleted_stock'].fillna(0)

# # Calculate adjusted_mkt_cap
# eod_market_cap_pivot['adjusted_mkt_cap'] = eod_market_cap_pivot['close index market cap'].shift(1) - eod_market_cap_pivot['mkt_cap_deleted_stock']

# # Fill NaNs with 0 in the adjusted_mkt_cap column
# eod_market_cap_pivot['adjusted_mkt_cap'] = eod_market_cap_pivot['adjusted_mkt_cap'].fillna(0)

# # Calculate divisor
# eod_market_cap_pivot['divisor'] = eod_market_cap_pivot.apply(
#     lambda row: 1.00 if pd.isna(row['mkt_cap_deleted_stock']) or row['mkt_cap_deleted_stock'] == 0 else row['adjusted_mkt_cap'] / row['close index market cap'],
#     axis=1
# )

# # Calculate gross index level
# eod_market_cap_pivot['gross index level'] = eod_market_cap_pivot['close index market cap'] / eod_market_cap_pivot['divisor']

# # Calculate Index Value
# eod_market_cap_pivot['Index Value'] = eod_market_cap_pivot['gross index level'] / eod_market_cap_pivot['gross index level'].iloc[0] * 100

# # Display the DataFrame with the new columns
# eod_market_cap_pivot


In [10]:
# # Create the new columns
# eod_market_cap_pivot['gross_change_mcap'] = eod_market_cap_pivot['close index market cap'].diff().fillna(0)
# # Calculate mkt_cap_deleted_stock
# eod_market_cap_pivot['mkt_cap_deleted_stock'] = eod_market_cap_pivot.apply(
#     lambda row: row['close index market cap'] if pd.isna(row['close index market cap']) or row['close index market cap'] == 0 else 0,
#     axis=1
# )

# eod_market_cap_pivot['mkt_cap_deleted_stock'] = mkt_cap_deleted_stock.fillna(0)

# # Calculate adjusted_mkt_cap
# eod_market_cap_pivot['adjusted_mkt_cap'] = eod_market_cap_pivot['close index market cap'].shift(1) - eod_market_cap_pivot['mkt_cap_deleted_stock']
# # Calculate divisor
# eod_market_cap_pivot['divisor'] = eod_market_cap_pivot.apply(
#     lambda row: 1.00 if pd.isna(row['adjusted_mkt_cap']) or row['adjusted_mkt_cap'] == 0 else row['adjusted_mkt_cap'] / row['close index market cap'],
#     axis=1
# )
# # Calculate gross index level
# eod_market_cap_pivot['gross index level'] = eod_market_cap_pivot['close index market cap'] / eod_market_cap_pivot['divisor']

# # Calculate Index Value
# eod_market_cap_pivot['Index Value'] = eod_market_cap_pivot['gross index level'] / eod_market_cap_pivot['gross index level'].iloc[0] * 100

# # Display the DataFrame with the new columns
# eod_market_cap_pivot
